# PURPOSE OF THIS NOTEBOOK 

This notebook takes the data needed for the individual 22 models and trains all the models to predict which job the titles belong to once the first model predicts the job family they belong to. There will be specific challenges I will need to look through, one of these specifically being that I need to figure how to benchmark these models. 

In [1]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
from transformers import BertTokenizer, BertModel, BertConfig
from pathlib import Path
from torch import cuda
import torch

from CommonFunctions import CustomDataset, BERTClass, loss_fn
from sklearn import metrics 
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

C:\Users\FrostyBB\AppData\Local\Temp\ipykernel_4172\2196876261.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# Setting up for GPU 

device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
device

'cuda'

In [4]:
# Load in the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [5]:
# Set the parmeters of the models 
MAX_LEN = 175
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 30
LEARNING_RATE = 2e-5

# Training the models 

Just like generating the data, I would need to create a for loop that will call and train the models based on what I need to do. 

In [6]:
# import the training/testing data
test_df = pd.read_csv("../Data/MajorGroupTrainTestSplit/ONET_11_test_df.csv")
train_df = pd.read_csv("../Data/MajorGroupTrainTestSplit/ONET_11_train_df.csv")
label_df = pd.read_csv("../Data/label_df.csv")
onet_group_df = pd.read_csv("../Data/ONET_Group_list.csv")


In [7]:
onet_group_list = onet_group_df['# ONET_Group'].to_list()

In [8]:
# Making sure that the label column is actual lists instead of strings of lists
train_df['Label'] = train_df['Label'].apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])
test_df['Label'] = test_df['Label'].apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])

In [9]:
# Define the parameters of the data
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0}

test_params = {'batch_size': VALID_BATCH_SIZE,
               'shuffle': True,
               'num_workers': 0}

## Create the loop to train the data in bulk 

In [10]:
def train(epoch, model, training_loader, optimizer):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss= loss_fn(outputs, targets)
        if _%100==0:
            print(f'Epoch : {epoch}, Loss: {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return outputs

In [11]:
def validation(epoch, model, testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _,data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            
            outputs = model(ids, mask, token_type_ids)

            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
# Start the for loop
for group in onet_group_list:
    # Import the data
    train_df = pd.read_csv(f"../Data/MajorGroupTrainTestSplit/ONET_{group}_train_df.csv")
    test_df = pd.read_csv(f"../Data/MajorGroupTrainTestSplit/ONET_{group}_test_df.csv")

    # Ensuring that the label columns contain lists not strings of lists
    train_df['Label'] = train_df['Label'].apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])
    test_df['Label'] = test_df['Label'].apply(lambda s: [float(x.strip(' []')) for x in s.split(',')])

    # transform the training and testing data into the datasets needed for training 
    train_set = CustomDataset(train_df, tokenizer, MAX_LEN)
    test_set = CustomDataset(test_df, tokenizer, MAX_LEN)

    # Run the DataLoader on the train/test set pair
    training_loader = DataLoader(train_set, **train_params)
    testing_loader = DataLoader(test_set, **test_params)

    # load in the model
    model = BERTClass(len=train_df.Label.str.len()[0])
    model.to(device)

    # load in the optimizer
    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    
    for epoch in range(EPOCHS):
        output = train(epoch, model, training_loader, optimizer)
        outputs, targets = validation(epoch, model, testing_loader)
        outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(targets, outputs)
        f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
        f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
        print(f"Accuracy Score = {accuracy}")
        print(f"F1 Score (Micro) = {f1_score_micro}")
        print(f"F1 Score (Macro) = {f1_score_macro}")
        


    # Lastly save the model for later testing. 
    torch.save(model, f'../Data/Models/ONET_Group_{group}_Model')

    # print the model number to know which model was trained 
    print(f"Model: {group}")

## Testing

In [63]:
def predict(input, tokenizer, model, device, label_list):
    '''
    GOAL:
    To take in an input, run the first model to predict the family that the occupation belongs too and then predict the occupation
    based on the family prediction
    '''
    # maniputlate the output to be ready for the model 
    input = " ".join(input.split())
    inputs = tokenizer.encode_plus(
        input,
        None,
        add_special_tokens=True,
        max_length=175,
        pad_to_max_length=True,
        return_token_type_ids=True
    )

    ids = torch.tensor(inputs['input_ids'], dtype=torch.long).to(device, dtype=torch.long)
    mask = torch.tensor(inputs['attention_mask'], dtype=torch.long).to(device, dtype=torch.long)
    token_type_ids = torch.tensor(inputs['token_type_ids'], dtype=torch.long).to(device, dtype=torch.long)

    # Import the model family model
    family_model = torch.load('../Data/Models/ONET_Family_Model')
    family_model.eval()

    # Run through the original model to guess family
    onet_family = family_model(ids.unsqueeze(0), mask.unsqueeze(0), token_type_ids.unsqueeze(0))

    # Extract prediction
    onet_family_df = pd.DataFrame(onet_family.cpu().detach().numpy().tolist(), columns=onet_group_list)
    family = onet_family_df.idxmax(axis='columns')[0]
    # Load in the specific model for the family that was trained. 
    job_predict_model = torch.load(f'../Data/Models/ONET_Group_{family}_Model')

    # Predict the job
    onet_job = job_predict_model(ids.unsqueeze(0), mask.unsqueeze(0), token_type_ids.unsqueeze(0))

    # Extract Prediction 
    onet_list = list(filter(lambda x: x.startswith(f"{family}"), label_list))
    
    onet_job_df = pd.DataFrame(onet_job.cpu().detach().numpy().tolist(), columns=onet_list)
    onet = onet_job_df.idxmax(axis='columns')

    return onet

   

In [75]:
input='Store Clerk'
output = predict(input, tokenizer, model, device, label_list=label_df.columns.to_list())

C:\Users\FrostyBB\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [76]:
output

0    41-2031.00
dtype: object